In [1]:
## Import the required libraries

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: \ 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.3.1=py37_0
  - defaults/linux-64::astropy==3.0.4=py37h14c3975_0
  - defaults/linux-64::bkcharts==0.2=py37_0
  - defaults/linux-64::blaze==0.11.3=py37_0
  - defaults/linux-64::bokeh==0.13.0=py37_0
  - defaults/linux-64::bottleneck==1.2.1=py37h035aef0_1
  - defaults/linux-64::dask==0.19.1=py37_0
  - defaults/linux-64::datashape==0.5.4=py37_1
  - defaults/linux-64::mkl-service==1.1.2=py37h90e4bf4_5
  - defaults/linux-64::numba==0.39.0=py37h04863e7_0
  - defaults/linux-64::numexpr==2.6.8=py37hd89afb7_0
  - defaults/linux-64::odo==0.5.1=py37_0
  - defaults/linux-64::pytables==3.4.4=py37ha205bf6_0
  - defaults/linux-64::pytest-arraydiff==0.2=py37h39e3cac_0
  - defaults/linux-64::pytest-astropy==0.4.0=py37_0
  - defaults/linux-64::pytest-doctestplus==0.1.3=py37_0
  - defaults/linux-64::pywavelets==1.0.0=py37h

In [2]:
# Create df Washington, DC neighborhoods and their lat and longs

df_dc = pd.read_csv('DC_Neighborhood_Labels.csv')
df_dc.head()

,LATITUDE,LONGITUDE,OBJECTID,GIS_ID,NAME,WEB_URL,LABEL_NAME,DATELASTMODIFIED
0,38.855658,-76.980348,1,nhood_050,Fort Stanton,http://NeighborhoodAction.dc.gov,Fort Stanton,2003-04-10T00:00:00.000Z
1,38.841077,-76.997950,2,nhood_031,Congress Heights,http://NeighborhoodAction.dc.gov,Congress Heights,2003-04-10T00:00:00.000Z
2,38.830237,-76.995636,3,nhood_123,Washington Highlands,http://NeighborhoodAction.dc.gov,Washington Highlands,2003-04-10T00:00:00.000Z
3,38.826952,-77.009271,4,nhood_008,Bellevue,http://NeighborhoodAction.dc.gov,Bellevue,2003-04-10T00:00:00.000Z
4,38.853688,-76.967660,5,nhood_073,Knox Hill/Buena Vista,http://NeighborhoodAction.dc.gov,Knox Hill/Buena Vista,2003-04-10T00:00:00.000Z


In [3]:
df_dc.shape

(132, 8)

In [4]:
# delete unnecessary colums

del df_dc['OBJECTID']
del df_dc['GIS_ID']
del df_dc['WEB_URL']
del df_dc['LABEL_NAME']
del df_dc['DATELASTMODIFIED']
df_dc.head()

,LATITUDE,LONGITUDE,NAME
0,38.855658,-76.980348,Fort Stanton
1,38.841077,-76.997950,Congress Heights
2,38.830237,-76.995636,Washington Highlands
3,38.826952,-77.009271,Bellevue
4,38.853688,-76.967660,Knox Hill/Buena Vista


In [5]:
# rename NAME column to NEIGHBORHOOD

df_dc.rename({'NAME':'NEIGHBORHOOD'}, axis = 'columns', inplace = True)

In [6]:
df_dc.head()

,LATITUDE,LONGITUDE,NEIGHBORHOOD
0,38.855658,-76.980348,Fort Stanton
1,38.841077,-76.997950,Congress Heights
2,38.830237,-76.995636,Washington Highlands
3,38.826952,-77.009271,Bellevue
4,38.853688,-76.967660,Knox Hill/Buena Vista


In [7]:
# Get the lat and long for DC

address = 'Washington, DC'

geolocator = Nominatim(user_agent="dc_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington, DC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington, DC are 38.8950092, -77.0365625.


In [8]:
# create map of DC and its neighborhoods using latitude and longitude values
map_dc = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(df_dc['LATITUDE'], df_dc['LONGITUDE'], df_dc['NEIGHBORHOOD']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dc)  
    
map_dc

In [9]:
# get foursquare credentials

CLIENT_ID = 'G02NF33I3VJ2ZOE4AL3CXWTV2TYEGUWIFUT4BFKX3DZVTBLZ' # your Foursquare ID
CLIENT_SECRET = 'CMCQG3SCSN4WOQ2RHZ5UQLJJ0WLOVRWKOIX4DPLWSMYBG1Z4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G02NF33I3VJ2ZOE4AL3CXWTV2TYEGUWIFUT4BFKX3DZVTBLZ
CLIENT_SECRET:CMCQG3SCSN4WOQ2RHZ5UQLJJ0WLOVRWKOIX4DPLWSMYBG1Z4


In [10]:
# checking to make sure that I can pull the lat and long for a specific neighborhood from the df_dc dataframe

neighborhood_latitude = df_dc.loc[0, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude = df_dc.loc[0, 'LONGITUDE'] # neighborhood longitude value

neighborhood_name = df_dc.loc[0, 'NEIGHBORHOOD'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Fort Stanton are 38.85565773, -76.98034771.


In [11]:
# function to repeat the exploring process to all the neighborhoods in DC
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):
            #print(name)

            # create the API request URL
            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            # make the GET request
            response = requests.get(url).json()
            results = response["response"]['venues']

            # return only relevant information for each nearby venue
            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['NEIGHBORHOOD', 
                  'NEIGHBORHOOD LATITUDE', 
                  'NEIGHBORHOOD LONGITUDE', 
                  'VENUE', 
                  'VENUE LATITUDE', 
                  'VENUE LONGITUDE', 
                  'VENUE CATEGORY']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [12]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
CLIENT_ID = 'G02NF33I3VJ2ZOE4AL3CXWTV2TYEGUWIFUT4BFKX3DZVTBLZ' # your Foursquare ID
CLIENT_SECRET = 'CMCQG3SCSN4WOQ2RHZ5UQLJJ0WLOVRWKOIX4DPLWSMYBG1Z4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G02NF33I3VJ2ZOE4AL3CXWTV2TYEGUWIFUT4BFKX3DZVTBLZ
CLIENT_SECRET:CMCQG3SCSN4WOQ2RHZ5UQLJJ0WLOVRWKOIX4DPLWSMYBG1Z4


### Now we will explore the different medical and EMS services in all of the DC neighborhoods. We will run this call for hospitals, urgent care clincis, firestations, and police stations.

In [13]:
# Use category id 4bf58dd8d48988d196941735 to only get hospitals
dc_hospital = getNearbyVenues(names=df_dc['NEIGHBORHOOD'], latitudes=df_dc['LATITUDE'], longitudes=df_dc['LONGITUDE'], radius=1000, categoryIds='4bf58dd8d48988d196941735')
dc_hospital.head()


,NEIGHBORHOOD,NEIGHBORHOOD LATITUDE,NEIGHBORHOOD LONGITUDE,VENUE,VENUE LATITUDE,VENUE LONGITUDE,VENUE CATEGORY
0,Fort Stanton,38.855658,-76.980348,St Elizabeths Hospital,38.849297,-76.988809,Hospital
1,Congress Heights,38.841077,-76.997950,Congress Heights Senior Wellness Center,38.839839,-77.005264,Hospital
2,Congress Heights,38.841077,-76.997950,St Elizabeths Hospital,38.849297,-76.988809,Hospital
3,Washington Highlands,38.830237,-76.995636,United Medical Center,38.835728,-76.984487,Hospital
4,Bellevue,38.826952,-77.009271,SHW- Hadley,38.824655,-77.012575,Hospital


In [21]:
# create map of DC using latitude and longitude values
hospital_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, venue in zip(dc_hospital['VENUE LATITUDE'], dc_hospital['VENUE LONGITUDE'], dc_hospital['NEIGHBORHOOD'], dc_hospital['VENUE']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='red',
        line_opacity=0.2,
        fill_opacity=0.7,
        parse_html=False).add_to(hospital_map)  
    
hospital_map

In [15]:
# Use category id 56aa371be4b08b9a8d573526 to only get urgent care clinics
dc_urgentcare = getNearbyVenues(names=df_dc['NEIGHBORHOOD'], latitudes=df_dc['LATITUDE'], longitudes=df_dc['LONGITUDE'], radius=1000, categoryIds='56aa371be4b08b9a8d573526')
dc_urgentcare.head()

,NEIGHBORHOOD,NEIGHBORHOOD LATITUDE,NEIGHBORHOOD LONGITUDE,VENUE,VENUE LATITUDE,VENUE LONGITUDE,VENUE CATEGORY
0,Fairlawn,38.869159,-76.977452,Urgent Care Center,38.866032,-76.989372,Urgent Care Center
1,Barry Farm,38.859255,-76.997281,Urgent Care Center,38.866032,-76.989372,Urgent Care Center
2,Historic Anacostia,38.863186,-76.984678,Urgent Care Center,38.866032,-76.989372,Urgent Care Center
3,Cardozo/Shaw,38.917168,-77.027550,Medics USA,38.912613,-77.038839,Doctor's Office
4,Hillsdale,38.858423,-76.990561,Urgent Care Center,38.866032,-76.989372,Urgent Care Center


In [17]:
# create map of DC using latitude and longitude values
urgentcare_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, venue in zip(dc_urgentcare['VENUE LATITUDE'], dc_urgentcare['VENUE LONGITUDE'], dc_urgentcare['NEIGHBORHOOD'], dc_urgentcare['VENUE']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='green',
        line_opacity=0.2,
        fill_opacity=0.7,
        parse_html=False).add_to(urgentcare_map)  
    
urgentcare_map

In [19]:
# Use category id 4bf58dd8d48988d12e941735 to only get police stations
dc_police = getNearbyVenues(names=df_dc['NEIGHBORHOOD'], latitudes=df_dc['LATITUDE'], longitudes=df_dc['LONGITUDE'], radius=1000, categoryIds='4bf58dd8d48988d12e941735')
dc_police.head()

,NEIGHBORHOOD,NEIGHBORHOOD LATITUDE,NEIGHBORHOOD LONGITUDE,VENUE,VENUE LATITUDE,VENUE LONGITUDE,VENUE CATEGORY
0,Fort Stanton,38.855658,-76.980348,Metropolitan Police Department - 7th District,38.853651,-76.969364,Police Station
1,Fort Stanton,38.855658,-76.980348,Sixth District Satellite Station,38.859010,-76.967216,Police Station
2,Fort Stanton,38.855658,-76.980348,MPD 7D Gym,38.854384,-76.970002,Police Station
3,Congress Heights,38.841077,-76.997950,7d Police Department,38.837091,-77.004633,Police Station
4,Congress Heights,38.841077,-76.997950,US Capitol Police Off-Site Screening,38.838776,-77.008734,Police Station


In [20]:
# create map of DC using latitude and longitude values
police_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, venue in zip(dc_police['VENUE LATITUDE'], dc_police['VENUE LONGITUDE'], dc_police['NEIGHBORHOOD'], dc_police['VENUE']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='yellow',
        line_opacity=0.2,
        fill_opacity=0.7,
        parse_html=False).add_to(police_map)  
    
police_map

In [22]:
# Use category id 4bf58dd8d48988d12c941735 to only get police stations
dc_fire = getNearbyVenues(names=df_dc['NEIGHBORHOOD'], latitudes=df_dc['LATITUDE'], longitudes=df_dc['LONGITUDE'], radius=1000, categoryIds='4bf58dd8d48988d12c941735')
dc_fire.head()

,NEIGHBORHOOD,NEIGHBORHOOD LATITUDE,NEIGHBORHOOD LONGITUDE,VENUE,VENUE LATITUDE,VENUE LONGITUDE,VENUE CATEGORY
0,Fort Stanton,38.855658,-76.980348,DCFD Engine 32 Truck 16,38.852665,-76.971353,Fire Station
1,Fort Stanton,38.855658,-76.980348,Firehouse Flop House (bunkroom),38.852697,-76.971519,Fire Station
2,Congress Heights,38.841077,-76.997950,DCFD ENGINE CO 25,38.844005,-77.000435,Fire Station
3,Washington Highlands,38.830237,-76.995636,DCFD Engine 33 / Truck 8,38.831036,-77.005139,Fire Station
4,Bellevue,38.826952,-77.009271,DCFD Engine 33 / Truck 8,38.831036,-77.005139,Fire Station


In [23]:
# create map of DC using latitude and longitude values
fire_map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood, venue in zip(dc_fire['VENUE LATITUDE'], dc_fire['VENUE LONGITUDE'], dc_fire['NEIGHBORHOOD'], dc_fire['VENUE']):
    label = '{}, {}'.format(venue, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='grey',
        line_opacity=0.2,
        fill_opacity=0.7,
        parse_html=False).add_to(fire_map)  
    
fire_map

## Now that we have all of the medical and EMS services in all of the Washington, DC neighborhoods, we will group the services by their neighborhoods in order to get the total count of each service in each neighborhood

In [24]:
# Group the venues by neighborhood
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('NEIGHBORHOOD').count()
    
    for n in startDf['NEIGHBORHOOD']:
        try:
            startDf.loc[startDf['NEIGHBORHOOD'] == n,columnTitle] = grouped.loc[n, 'VENUE']
        except:
            startDf.loc[startDf['NEIGHBORHOOD'] == n,columnTitle] = 0

In [33]:
# Creat df for neighborhoods and their count of venues
dc_data = df_dc.copy()
addColumn(dc_data, 'HOSPITAL', dc_hospital)
addColumn(dc_data, 'URGENT CARE', dc_urgentcare)
addColumn(dc_data, 'POLICE STATION', dc_police)
addColumn(dc_data, 'FIRE STATION', dc_fire)
dc_data.head()

,LATITUDE,LONGITUDE,NEIGHBORHOOD,HOSPITAL,URGENT CARE,POLICE STATION,FIRE STATION
0,38.855658,-76.980348,Fort Stanton,1.0,0.0,3.0,2.0
1,38.841077,-76.997950,Congress Heights,2.0,0.0,2.0,1.0
2,38.830237,-76.995636,Washington Highlands,1.0,0.0,0.0,1.0
3,38.826952,-77.009271,Bellevue,1.0,0.0,2.0,2.0
4,38.853688,-76.967660,Knox Hill/Buena Vista,0.0,0.0,4.0,2.0


## Using the dc_data df, we will apply weights to each of the services

In [28]:
# This cell will weight the neighborhoods based on the values below

# weighted at 2 due to the importance and versitility of hospitals
weight_hospital = 2

# weighted at 2 due to convenience 
weight_urgentcare = 2

# weighted at 1.5 due to fire stations generally having ambulance/paramedic/emt services
weight_firestation = 1.5

# weighted lowest due to lack of medical serivces able to provide
weight_policestation = 1

In [27]:
dc_weighted = dc_data[['NEIGHBORHOOD']].copy()

## The below df shows the weighted scores of each neighborhood in Washington, DC.

In [29]:
dc_weighted['Score'] = dc_data['HOSPITAL'] * weight_hospital + dc_data['URGENT CARE'] * weight_urgentcare + dc_data['FIRE STATION'] * weight_firestation + dc_data['POLICE STATION'] * weight_policestation
dc_weighted = dc_weighted.sort_values(by=['Score'], ascending=False)
dc_weighted

,NEIGHBORHOOD,Score
35,Pleasant Plains,107.5
109,Park View,106.0
82,West End,105.5
81,Foggy Bottom,101.5
79,Connecticut Avenue/K Street,99.5
111,Howard University,97.5
80,George Washington University,93.0
40,Stronghold,91.5
63,Bloomingdale,49.5
43,Downtown East,39.0


## Pleasant Plains has the highest score of all the neighborhoods in Washington, DC. Lets explore this neighborhood more

In [49]:
neighborhood_latitude = df_dc.loc[35, 'LATITUDE'] # neighborhood latitude value
neighborhood_longitude = df_dc.loc[35, 'LONGITUDE'] # neighborhood longitude value

neighborhood_name = df_dc.loc[35, 'NEIGHBORHOOD'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Pleasant Plains are 38.92947297, -77.023101.


In [52]:
# function to add markers for given venues to map
def addToMap(df, color, existingMap):
    for lat, lng, neighborhood, venue, venueCat in zip(df['VENUE LATITUDE'], df['VENUE LONGITUDE'], df['NEIGHBORHOOD'], df['VENUE'], df['VENUE CATEGORY']):
        label = '{} ({}) - {}'.format(venue, venueCat, neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [53]:
# Map of Pleasant Plains and the medical / EMS services within 1000m of the neighborhoods center

pp_result_map = folium.Map(location=[neighborhood_latitude, neighborhood_longitude], zoom_start=15)

top_neighborhood = df_dc[df_dc['NEIGHBORHOOD'] == 'Pleasant Plains'] 

for lat, lng, neighborhood in zip(top_neighborhood['LATITUDE'], top_neighborhood['LONGITUDE'], top_neighborhood['NEIGHBORHOOD']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(pp_result_map) 

addToMap(dc_hospital[dc_hospital['NEIGHBORHOOD'] == 'Pleasant Plains'], 'red', pp_result_map)
addToMap(dc_urgentcare[dc_urgentcare['NEIGHBORHOOD'] == 'Pleasant Plains'], 'green', pp_result_map)
addToMap(dc_fire[dc_fire['NEIGHBORHOOD'] == 'Pleasant Plains'], 'grey', pp_result_map)
addToMap(dc_police[dc_police['NEIGHBORHOOD'] == 'Pleasant Plains'], 'yellow', pp_result_map)

pp_result_map

## It's clear to see that Pleasant Plains has the highest weighted score due to its close proximity MedStar hospital, the largest private hospital in Washington, DC. There are also 5 fire stations and 5 police stations within 1000m of the center of the Pleasant Plains neighborhood